This function enables patterns generated via Ernould's method to be written to a .up2 file to enable processing by ATEX 

In [1]:
import numpy as np
import os
import sys

import ErnouldsMethod
#import write_up2
import conversions
import write_up2 as write_up2

In [2]:
Ernould_PC = np.array([600, 600, 16*1000/20]) #the pattern center to be used for Ernould's method
detector_shape = (1200,1200)

filepath = '/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/'
outputpath = '/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/Inputs/PCsensitivity.up2'

In [3]:
numPats = 5*8*12  #number of patterns to simulate
print("Number of patterns to simulate: ", numPats)
options = ['AltPCx', 'AltPCy', 'AltDD', 'AltPCxPCy', 'AltPCxPCyDD']
cases = ['Case1', 'Case2', 'Case3', 'Case4', 'Case5', 'Case6', 'Case7', 'Case8']

Number of patterns to simulate:  480


In [4]:
pattern_filenames = []

for option in options:
    for case in cases:
        #generate the string for the filename

        filename = filepath + 'EBSD_' + option + '_' + case + '_Jan52026.h5'
        pattern_filenames.append(filename)
        print("Added pattern filename: ", filename)




Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case1_Jan52026.h5
Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case2_Jan52026.h5
Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case3_Jan52026.h5
Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case4_Jan52026.h5
Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case5_Jan52026.h5
Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case6_Jan52026.h5
Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case7_Jan52026.h5
Added pattern filename:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case8_Jan5

Reading in the .csv file which contains all 12 of the strain cases

In [5]:
# EMEBSDfilename = '/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/Inputs/EBSDpattern_Al_July222025.h5'
Testcasesfilename = '/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/Inputs/PC_Sensitivity.csv'

# outputfile = '/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/Inputs/E11_notNormalized_Dec42025.up2'
# EMEBSD_pattern = ErnouldsMethod.read_EMEBSD(EMEBSDfilename)

# #determine the reference pattern coordinates need to pass in PC and detector shape
# ref_coords = ErnouldsMethod.pattern_coords(Ernould_PC, detector_shape)
#             #Read in the test cases from a csv file
#             #load in the testcases
testcases_data = np.genfromtxt(Testcasesfilename, delimiter=',', encoding='utf-8-sig')
            #breaking the testcases data into euler angles and strain/rotation components
        
w = testcases_data[:, 0:3]
e = testcases_data[:, 3:]

print ("Number of test cases loaded: ", w.shape[0])

#create the tensor Fe_values
Fe_values = np.zeros((w.shape[0], 3, 3), dtype=np.float32)
#calculate the Fe values for each test case
for i in range(w.shape[0]):
    Fe_values[i, :, :] = ErnouldsMethod.determineF(e[i], w[i])
    #normalize so that h33 = 1
    #Fe_values[i, :, :] = Fe_values[i, :, :]/Fe_values[i, 2, 2]

h = conversions.F2h(Fe_values, [0, 0, 16*1000/20])



Number of test cases loaded:  12


In [ ]:

counter = 0 
generated_patterns = np.zeros((numPats, detector_shape[0], detector_shape[1]), dtype=np.float32)

#for each of the filenames generate a pattern at each of the h values
for filename in pattern_filenames:
    print("Generating patterns for file: ", filename)
    EMEBSD_pattern = ErnouldsMethod.read_EMEBSD(filename)
    ref_coords = ErnouldsMethod.pattern_coords(Ernould_PC, detector_shape)
    for i in range(w.shape[0]):
        print("Generating pattern ", counter+1, " of ", numPats)
        Fe = Fe_values[i, :, :]
        generated_patterns[counter, :, :] = ErnouldsMethod.generate_patterns(Fe, Ernould_PC, ref_coords, EMEBSD_pattern)
        counter += 1
    #write the patterns to the generated patterns array




Generating patterns for file:  /Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/AlteredPC-Jan5/EBSD_AltPCx_Case1_Jan52026.h5
Generating pattern  1  of  480
Generating pattern  2  of  480
Generating pattern  3  of  480


KeyboardInterrupt: 

In [ ]:
#save every tenth pattern as a .png file to check
import matplotlib.pyplot as plt

#folder to save the images
output_folder = '/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/Inputs/Patterns'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for i in range(0, numPats, 10):
    plt.imsave(os.path.join(output_folder, f'pattern_{i:03d}.png'), generated_patterns[i], cmap='gray')
    

In [ ]:

write_up2.write_up2(generated_patterns, outputpath, bit_depth=16)

/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/write_up2.py:136: RuntimeWarning: invalid value encountered in divide
  pats_array = np.around((pats_array - mns) / (mxs - mns) * 65535).astype(
/Users/crestiennedechaine/Scripts/DIC-HREBSD/DIC-HREBSD/write_up2.py:136: RuntimeWarning: invalid value encountered in cast
  pats_array = np.around((pats_array - mns) / (mxs - mns) * 65535).astype(
